# 03 - FLAML 优化技巧

> 🎯 学习目标：掌握微软FLAML的快速优化能力，理解成本敏感的超参数调优

---

## 📚 本节内容

1. FLAML 简介与核心优势
2. 快速上手
3. 成本敏感优化
4. 大数据集优化
5. 与其他AutoML工具对比
6. 高级配置

---

## ⚡ 为什么选择 FLAML？

**FLAML = Fast and Lightweight AutoML**

### 核心优势

1. **速度最快** ⚡
   - 比传统AutoML快10-100倍
   - 低计算成本

2. **资源高效** 💰
   - 成本敏感的优化算法
   - 适合预算有限的场景

3. **大数据友好** 📈
   - 支持大规模数据集
   - 增量学习

4. **工业级** 🏢
   - 微软内部大规模使用
   - 生产环境验证

---

## 1. 安装与导入

In [ ]:
# 安装 FLAML
# !pip install flaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

print("✅ 基础库导入成功")

---

## 2. 快速上手：分类任务

### 2.1 加载数据

In [ ]:
# 加载乳腺癌数据集
X, y = load_breast_cancer(return_X_y=True)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"训练集形状: {X_train.shape}")
print(f"测试集形状: {X_test.shape}")

### 2.2 使用 FLAML（最简单的方式）

In [ ]:
from flaml import AutoML

# 创建 AutoML 对象
automl = AutoML()

# 设置并训练
start_time = time.time()

automl.fit(
    X_train, y_train,
    task='classification',      # 任务类型
    time_budget=60,             # 时间预算（秒）
    metric='accuracy',          # 优化指标
    log_file_name='flaml.log',  # 日志文件
    seed=42
)

training_time = time.time() - start_time

print(f"\n✅ 训练完成！耗时: {training_time:.2f}秒")

### 2.3 查看最佳模型

In [ ]:
# 最佳模型信息
print("最佳模型:")
print(f"  模型类型: {automl.best_estimator}")
print(f"\n最佳配置:")
print(automl.best_config)
print(f"\n最佳损失: {automl.best_loss:.4f}")

### 2.4 预测与评估

In [ ]:
# 预测
y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)

# 评估
accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_pred_proba)

print(f"测试集准确率: {accuracy:.4f}")
print(f"测试集 Log Loss: {logloss:.4f}")

---

## 3. 回归任务示例

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score, mean_squared_error

# 加载数据
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# FLAML 回归
automl_reg = AutoML()
automl_reg.fit(
    X_train, y_train,
    task='regression',
    time_budget=60,
    metric='r2',
    seed=42
)

# 预测
y_pred = automl_reg.predict(X_test)

# 评估
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\nR² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"\n最佳模型: {automl_reg.best_estimator}")

---

## 4. 成本敏感优化（FLAML核心特性）

### 4.1 理解成本敏感优化

FLAML的核心算法：**CFO (Cost-Frugal Optimization)**

- 考虑训练成本和模型性能的权衡
- 优先搜索"性价比"高的配置
- 避免浪费时间在低收益配置上

In [ ]:
# 对比：不同时间预算的效果
time_budgets = [30, 60, 120, 300]
results = []

for budget in time_budgets:
    automl_temp = AutoML()
    
    start = time.time()
    automl_temp.fit(
        X_train, y_train,
        task='classification',
        time_budget=budget,
        metric='accuracy',
        verbose=0,
        seed=42
    )
    actual_time = time.time() - start
    
    accuracy = accuracy_score(y_test, automl_temp.predict(X_test))
    
    results.append({
        'time_budget': budget,
        'actual_time': actual_time,
        'accuracy': accuracy,
        'best_model': automl_temp.best_estimator
    })

# 展示结果
results_df = pd.DataFrame(results)
print("\n时间预算 vs 准确率:")
print(results_df)

In [ ]:
# 可视化：时间 vs 准确率
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(results_df['time_budget'], results_df['accuracy'], 'o-')
plt.xlabel('Time Budget (seconds)')
plt.ylabel('Accuracy')
plt.title('Time Budget vs Accuracy')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(results_df['actual_time'], results_df['accuracy'], 's-', color='orange')
plt.xlabel('Actual Time (seconds)')
plt.ylabel('Accuracy')
plt.title('Actual Time vs Accuracy')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5. 高级配置

### 5.1 限制模型类型

In [ ]:
# 只使用树模型
automl_trees = AutoML()
automl_trees.fit(
    X_train, y_train,
    task='classification',
    time_budget=60,
    estimator_list=['xgboost', 'rf', 'lgbm'],  # 限制模型类型
    seed=42
)

print(f"最佳模型: {automl_trees.best_estimator}")
print(f"准确率: {accuracy_score(y_test, automl_trees.predict(X_test)):.4f}")

### 5.2 自定义评估指标

In [ ]:
from sklearn.metrics import f1_score, make_scorer

# 自定义指标函数
def custom_metric(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

# 使用自定义指标
automl_custom = AutoML()
automl_custom.fit(
    X_train, y_train,
    task='classification',
    time_budget=60,
    metric=custom_metric,
    seed=42
)

print(f"\n最佳F1 Score: {automl_custom.best_loss:.4f}")

### 5.3 早停策略

In [ ]:
# 设置早停：如果性能不再提升，提前结束
automl_early = AutoML()
automl_early.fit(
    X_train, y_train,
    task='classification',
    time_budget=300,
    early_stop=True,               # 启用早停
    seed=42
)

print("✅ 使用早停策略训练完成")

### 5.4 交叉验证设置

In [ ]:
# 自定义交叉验证折数
automl_cv = AutoML()
automl_cv.fit(
    X_train, y_train,
    task='classification',
    time_budget=60,
    n_splits=10,  # 10折交叉验证
    seed=42
)

print(f"使用 {automl_cv.best_config_per_estimator} 折交叉验证")

---

## 6. FLAML vs 其他 AutoML 工具对比

### 6.1 速度对比

In [ ]:
# 对比 FLAML 和 sklearn GridSearch 的速度
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# FLAML
start = time.time()
automl_fast = AutoML()
automl_fast.fit(X_train, y_train, task='classification', time_budget=60, seed=42)
flaml_time = time.time() - start
flaml_acc = accuracy_score(y_test, automl_fast.predict(X_test))

# sklearn GridSearchCV
start = time.time()
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}
grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid.fit(X_train, y_train)
grid_time = time.time() - start
grid_acc = accuracy_score(y_test, grid.predict(X_test))

# 对比结果
print("\n速度对比:")
print(f"FLAML:       {flaml_time:.2f}秒, 准确率: {flaml_acc:.4f}")
print(f"GridSearch:  {grid_time:.2f}秒, 准确率: {grid_acc:.4f}")
print(f"\nFLAML 速度提升: {grid_time/flaml_time:.2f}x")

---

## 7. 大数据集优化技巧

### 7.1 使用采样策略

In [ ]:
# 对于大数据集，FLAML 会自动采样
automl_large = AutoML()
automl_large.fit(
    X_train, y_train,
    task='classification',
    time_budget=60,
    max_iter=None,           # 不限制迭代次数
    sample=True,             # 启用采样
    seed=42
)

print("✅ 大数据集优化完成")

### 7.2 使用增量学习

In [ ]:
# FLAML 支持增量学习（对于支持的模型）
automl_incremental = AutoML()
automl_incremental.fit(
    X_train, y_train,
    task='classification',
    time_budget=60,
    estimator_list=['sgd'],  # 使用支持增量学习的模型
    seed=42
)

print("✅ 增量学习完成")

---

## 8. 模型保存与部署

In [ ]:
import pickle

# 保存模型
with open('flaml_model.pkl', 'wb') as f:
    pickle.dump(automl, f)

# 加载模型
with open('flaml_model.pkl', 'rb') as f:
    loaded_automl = pickle.load(f)

# 预测
predictions = loaded_automl.predict(X_test)

print("✅ 模型保存和加载成功")
print(f"加载后准确率: {accuracy_score(y_test, predictions):.4f}")

---

## 9. 总结与最佳实践

### FLAML 核心优势

1. **速度**: 成本敏感优化，快速找到好模型
2. **效率**: 低计算成本，适合预算有限场景
3. **可扩展**: 支持大数据集
4. **工业级**: 微软生产环境验证

### 使用场景

```python
# 快速验证（1分钟）
automl.fit(..., time_budget=60)

# 标准训练（5-10分钟）
automl.fit(..., time_budget=300)

# 深度搜索（30分钟-1小时）
automl.fit(..., time_budget=3600)
```

### 最佳实践

1. **时间预算**: 根据项目时间合理设置
2. **模型选择**: 限制模型类型可以加速搜索
3. **早停**: 对于简单任务，启用早停节省时间
4. **大数据集**: 使用采样策略
5. **日志分析**: 查看 `flaml.log` 了解搜索过程

### 优势与局限

**优势**：
- ✅ 速度最快
- ✅ 成本最低
- ✅ 大数据友好

**局限**：
- ❌ 功能不如PyCaret丰富
- ❌ 可视化较少
- ❌ 集成功能有限

---

## 🎯 练习任务

1. **任务1**: 对比不同时间预算下的性能
2. **任务2**: 理解成本敏感优化的原理
3. **任务3**: 在大数据集上测试FLAML
4. **任务4**: 对比FLAML和PyCaret的效果

---

## 📚 参考资源

- [FLAML 官方文档](https://microsoft.github.io/FLAML/)
- [FLAML 论文](https://arxiv.org/abs/1911.04706)
- [微软研究院博客](https://www.microsoft.com/en-us/research/project/fast-and-lightweight-automl/)

---

**下一节**: `04_h2o_automl.ipynb` - H2O AutoML 企业级应用